In [ ]:
# !pip install monai pycuda einops

## Create a model and export it as ONNX

In [ ]:
import monai
import torch

batch_size = 1
device = 'cuda'
model = monai.networks.nets.SegResNet(spatial_dims=3,
                                      in_channels=1,
                                      out_channels=3,
                                      init_filters=24).to(device)

After training...

In [ ]:
torch.save(model.state_dict(), './models/seg.pt')

In [ ]:
dummy_input = torch.randn(batch_size, 1, 224, 224, 128).to(device)

with torch.no_grad():
    print(model(dummy_input).shape)

torch.onnx.export(model, dummy_input, './models/seg.onnx')

## Convert the ONNX model to TensorRT engine

In [ ]:
# Restart notebook kernel
import os
os._exit(00)

- Convert to FP32 engine

In [ ]:
!trtexec --onnx=./models/seg.onnx --saveEngine=./models/seg_32.engine 

- Convert to FP16 engine

In [ ]:
!trtexec --onnx=./models/seg.onnx --saveEngine=./models/seg_16.engine --fp16 

## Benchmark

In [1]:
from benchmark import NativeTorchBenchmark, TensorRTBehcnmark
import numpy as np
import monai

n_infers = 100
batch_size = 1
input_image = np.random.normal(size=[batch_size, 1, 224, 224, 128])

- Native PyTorch

In [2]:
model = monai.networks.nets.SegResNet(spatial_dims=3,
                                      in_channels=1,
                                      out_channels=3,
                                      init_filters=24)

pt_bm = NativeTorchBenchmark(n_infers=n_infers,
                             batch_size=batch_size,
                             samples=input_image,
                             model_arch=model,
                             model_ckpt='./models/seg.pt')

In [3]:
pt_bm.benchmark()

100%|████████████████████████████████████████████████████████| 100/100 [00:56<00:00,  1.77it/s]

Throughputs (Native PyTorch): 1.7743


- TensorRT FP32

In [4]:
trt_bm_fp32 = TensorRTBehcnmark(n_infers=n_infers,
                                batch_size=batch_size,
                                samples=input_image,
                                engine_path='./models/seg_32.engine')

In [5]:
trt_bm_fp32.benchmark()

100%|████████████████████████████████████████████████████████| 100/100 [00:39<00:00,  2.53it/s]

Throughputs: 2.5334


- TensorRT FP16

In [6]:
trt_bm_fp16 = TensorRTBehcnmark(n_infers=n_infers,
                                batch_size=batch_size,
                                samples=input_image,
                                engine_path='./models/seg_16.engine')

In [7]:
trt_bm_fp16.benchmark()

100%|████████████████████████████████████████████████████████| 100/100 [00:22<00:00,  4.54it/s]

Throughputs: 4.543
